<a href="https://colab.research.google.com/github/michael-borck/weatherwise-template/blob/main/starter_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🌦️ WeatherWise – Starter Notebook

Welcome to your **WeatherWise** project notebook! This scaffold is designed to help you build your weather advisor app using Python, visualisations, and AI-enhanced development.

---

📄 **Full Assignment Specification**  
See [`ASSIGNMENT.md`](ASSIGNMENT.md) or check the LMS for full details.

📝 **Quick Refresher**  
A one-page summary is available in [`resources/assignment-summary.md`](resources/assignment-summary.md).

---

🧠 **This Notebook Structure is Optional**  
You’re encouraged to reorganise, rename sections, or remove scaffold cells if you prefer — as long as your final version meets the requirements.

✅ You may delete this note before submission.



## 🧰 Setup and Imports

This section imports commonly used packages and installs any additional tools used in the project.

- You may not need all of these unless you're using specific features (e.g. visualisations, advanced prompting).
- The notebook assumes the following packages are **pre-installed** in the provided environment or installable via pip:
  - `requests`, `matplotlib`, `pyinputplus`
  - `fetch-my-weather` (for accessing weather data easily)
  - `hands-on-ai` (for AI logging, comparisons, or prompting tools)

If you're running this notebook in **Google Colab**, uncomment the following lines to install the required packages.


In [ ]:
# 🧪 Optional packages — uncomment if needed in Colab or JupyterHub
!pip install fetch-my-weather
!pip install hands-on-ai


In [ ]:
import os

#  Testing ai api key: student-api-key-123
os.environ['HANDS_ON_AI_SERVER'] = 'http://ollama.serveur.au'
os.environ['HANDS_ON_AI_MODEL'] = 'granite3.2'
os.environ['HANDS_ON_AI_API_KEY'] = input('Enter your API key: ')

## 📦 Setup and Configuration
Import required packages and setup environment.

In [ ]:
import requests
import matplotlib.pyplot as plt
import pyinputplus as pyip
# ✅ Import after installing (if needed)
from fetch_my_weather import get_weather
from hands_on_ai.chat import get_response

# Add any other setup code here

import json
import re
from datetime import datetime, timedelta

## 🌤️ Weather Data Functions

In [ ]:
# Define get_weather_data() function here
def safe_int(value, default=0):
    """Safely convert value to int, return default if conversion fails."""
    if value is None:
        return default
    try:
        return int(value)
    except (ValueError, TypeError):
        return default

def safe_float(value, default=0.0):
    """Safely convert value to float, return default if conversion fails."""
    if value is None:
        return default
    try:
        return float(value)
    except (ValueError, TypeError):
        return default

def safe_get_attr(obj, attr_path, default=None):
    """Safely get nested attribute, return default if any step fails."""
    try:
        attrs = attr_path.split('.')
        result = obj
        for attr in attrs:
            if hasattr(result, attr):
                result = getattr(result, attr)
            else:
                return default
        return result
    except:
        return default



# Update the weather data processing section in get_weather_data()
def get_weather_data(location, forecast_days=5):
    """
    Updated version with better error handling
    """
    if not isinstance(forecast_days, int):
        try:
            forecast_days = int(forecast_days)
            if not (1 <= forecast_days <= 5):
                print(f"Warning: forecast_days ({forecast_days}) out of range (1-5). Clamping.")
        except ValueError:
            print(f"Warning: forecast_days ('{forecast_days}') is not a valid integer. Defaulting to 1.")
            forecast_days = 1

    if forecast_days < 1:
        forecast_days = 1
    elif forecast_days > 5:
        forecast_days = 5

    processed_data = {"current": None, "forecast": [], "error": None}

    try:
        raw_data = get_weather(location)
        
        if raw_data is None:
            processed_data["error"] = f"No data returned from weather service for '{location}'."
            return processed_data
        
        # Check for errors
        if hasattr(raw_data, 'error') and getattr(raw_data, 'error'):
            processed_data["error"] = raw_data.error
            return processed_data

        # Process current conditions with safe conversions
        if hasattr(raw_data, 'current_condition') and raw_data.current_condition:
            current = raw_data.current_condition[0]
            processed_data["current"] = {
                "temperature_c": safe_int(getattr(current, 'temp_C', None)),
                "feels_like_c": safe_int(getattr(current, 'FeelsLikeC', None)),
                "description": safe_get_attr(current, 'weatherDesc.0.value', "N/A"),
                "humidity_percent": safe_int(getattr(current, 'humidity', None)),
                "wind_kmph": safe_int(getattr(current, 'windspeedKmph', None)),
                "precipitation_mm": safe_float(getattr(current, 'precipMM', None))
            }
        else:
            processed_data["error"] = f"Current conditions data missing or malformed for '{location}'."
            return processed_data

        # Process forecast with safe conversions
        if hasattr(raw_data, 'weather') and raw_data.weather:
            full_forecast_list = raw_data.weather
            for i, day_data in enumerate(full_forecast_list):
                if i >= forecast_days:
                    break
                
                hourly_slots = getattr(day_data, 'hourly', [])
                representative_hourly_data = None
                if len(hourly_slots) > 4:
                    representative_hourly_data = hourly_slots[4]
                elif hourly_slots:
                    representative_hourly_data = hourly_slots[0]
                
                if representative_hourly_data:
                    forecast_entry = {
                        "date": getattr(day_data, 'date', 'N/A'),
                        "max_temp_c": safe_int(getattr(day_data, 'maxtempC', None)),
                        "min_temp_c": safe_int(getattr(day_data, 'mintempC', None)),
                        "avg_temp_c": safe_int(getattr(day_data, 'avgtempC', None)),
                        "description": safe_get_attr(representative_hourly_data, 'weatherDesc.0.value', "N/A"),
                        "chance_of_rain_percent": safe_int(getattr(representative_hourly_data, 'chanceofrain', None)),
                        "total_precip_mm": safe_float(getattr(representative_hourly_data, 'precipMM', None))
                    }
                    processed_data["forecast"].append(forecast_entry)
        
        if not processed_data["current"] and not processed_data["forecast"] and not processed_data["error"]:
             processed_data["error"] = f"Failed to parse valid weather data for '{location}' despite receiving a response."

    except Exception as e:
        processed_data["error"] = f"An unexpected error occurred fetching/processing weather for '{location}': {str(e)}"
        
    return processed_data

## 📊 Visualisation Functions

In [ ]:
# Define create_temperature_visualisation() and create_precipitation_visualisation() here
def create_temperature_visualisation(weather_data, output_type='display'):
    """
    Create a line chart visualisation of daily maximum and minimum temperature data.

    Args:
        weather_data (dict): The processed weather data, expected to have a 'forecast'
                             key with a list of daily forecast dictionaries. Each
                             dictionary should contain 'date', 'max_temp_c', and
                             'min_temp_c'.
        output_type (str): Either 'display' to show the plot directly,
                             or 'figure' to return the matplotlib figure object.

    Returns:
        If output_type is 'figure', returns the matplotlib.figure.Figure object.
        Otherwise (for 'display'), displays the plot and returns None.
    """
    if weather_data.get("error"):
        print(f"Cannot create visualisation: {weather_data['error']}")
        if output_type == 'figure':
            # Return an empty figure or None if error, to maintain return type consistency
            fig, ax = plt.subplots()
            ax.text(0.5, 0.5, "Error: No data to display.", ha='center', va='center')
            return fig
        return None

    forecast_list = weather_data.get("forecast")
    if not forecast_list:
        print("No forecast data available to create temperature visualisation.")
        if output_type == 'figure':
            fig, ax = plt.subplots()
            ax.text(0.5, 0.5, "No forecast data.", ha='center', va='center')
            return fig
        return None

    dates = []
    max_temps = []
    min_temps = []

    for day_data in forecast_list:
        try:
            # Attempt to parse date for cleaner x-axis labels, fallback to string
            date_obj = datetime.strptime(day_data.get("date"), "%Y-%m-%d")
            dates.append(date_obj.strftime("%b %d")) # e.g., "Oct 27"
        except (ValueError, TypeError):
            dates.append(day_data.get("date", "Unknown")) # Fallback if date is not as expected
            
        max_temps.append(day_data.get("max_temp_c"))
        min_temps.append(day_data.get("min_temp_c"))

    # Filter out None values if any field was missing from a forecast entry
    # This requires dates, max_temps, and min_temps to be filtered consistently
    valid_indices = [i for i, (mx, mn) in enumerate(zip(max_temps, min_temps)) if mx is not None and mn is not None]
    
    if not valid_indices:
        print("Insufficient temperature data in forecast to create visualisation.")
        if output_type == 'figure':
            fig, ax = plt.subplots()
            ax.text(0.5, 0.5, "Insufficient data.", ha='center', va='center')
            return fig
        return None
        
    dates = [dates[i] for i in valid_indices]
    max_temps = [max_temps[i] for i in valid_indices]
    min_temps = [min_temps[i] for i in valid_indices]

    if not dates: # Double check after filtering
        print("No valid dates found for temperature visualisation after filtering.")
        if output_type == 'figure':
            fig, ax = plt.subplots()
            ax.text(0.5, 0.5, "No valid dates.", ha='center', va='center')
            return fig
        return None

    fig, ax = plt.subplots(figsize=(10, 6)) # Create a figure and an axes

    ax.plot(dates, max_temps, marker='o', linestyle='-', color='r', label='Max Temp (°C)')
    ax.plot(dates, min_temps, marker='s', linestyle='--', color='b', label='Min Temp (°C)')

    ax.set_xlabel("Date")
    ax.set_ylabel("Temperature (°C)")
    ax.set_title("Daily Temperature Forecast")
    ax.legend() # Show legend
    ax.grid(True, linestyle=':', alpha=0.7) # Add a grid for readability

    plt.xticks(rotation=45, ha="right") # Rotate date labels for better fit
    plt.tight_layout() # Adjust plot to prevent labels from being cut off

    if output_type == 'figure':
        return fig
    elif output_type == 'display':
        plt.show()
        return None
    else:
        print(f"Warning: Unknown output_type '{output_type}'. Displaying plot by default.")
        plt.show()
        return None


In [ ]:

def create_precipitation_visualisation(weather_data, output_type='display'):
    """
    Create a bar chart visualisation of daily chance of rain.

    Args:
        weather_data (dict): The processed weather data, expected to have a 'forecast'
                             key with a list of daily forecast dictionaries. Each
                             dictionary should contain 'date' and 'chance_of_rain_percent'.
        output_type (str): Either 'display' to show the plot directly,
                             or 'figure' to return the matplotlib figure object.

    Returns:
        If output_type is 'figure', returns the matplotlib.figure.Figure object.
        Otherwise (for 'display'), displays the plot and returns None.
    """
    if weather_data.get("error"):
        print(f"Cannot create precipitation visualisation: {weather_data['error']}")
        if output_type == 'figure':
            fig, ax = plt.subplots()
            ax.text(0.5, 0.5, "Error: No data to display.", ha='center', va='center')
            return fig
        return None

    forecast_list = weather_data.get("forecast")
    if not forecast_list:
        print("No forecast data available to create precipitation visualisation.")
        if output_type == 'figure':
            fig, ax = plt.subplots()
            ax.text(0.5, 0.5, "No forecast data.", ha='center', va='center')
            return fig
        return None

    dates = []
    chance_of_rain = []

    for day_data in forecast_list:
        try:
            date_obj = datetime.strptime(day_data.get("date"), "%Y-%m-%d")
            dates.append(date_obj.strftime("%b %d")) # e.g., "Oct 27"
        except (ValueError, TypeError):
            dates.append(day_data.get("date", "Unknown"))
            
        chance_of_rain.append(day_data.get("chance_of_rain_percent"))

    # Filter out None values for chance_of_rain, keeping dates and precip consistent
    valid_indices = [i for i, rain_chance in enumerate(chance_of_rain) if rain_chance is not None]
    
    if not valid_indices:
        print("Insufficient precipitation data in forecast to create visualisation.")
        if output_type == 'figure':
            fig, ax = plt.subplots()
            ax.text(0.5, 0.5, "Insufficient data.", ha='center', va='center')
            return fig
        return None

    dates = [dates[i] for i in valid_indices]
    chance_of_rain = [chance_of_rain[i] for i in valid_indices]
    
    if not dates: # Double check after filtering
        print("No valid dates found for precipitation visualisation after filtering.")
        if output_type == 'figure':
            fig, ax = plt.subplots()
            ax.text(0.5, 0.5, "No valid dates.", ha='center', va='center')
            return fig
        return None

    fig, ax = plt.subplots(figsize=(10, 6)) # Create a figure and an axes

    bars = ax.bar(dates, chance_of_rain, color='skyblue', label='Chance of Rain (%)')

    # Add text labels on top of each bar
    for bar in bars:
        yval = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2.0, yval + 1, f'{yval}%', ha='center', va='bottom')

    ax.set_xlabel("Date")
    ax.set_ylabel("Chance of Rain (%)")
    ax.set_title("Daily Precipitation Forecast (Chance of Rain)")
    ax.set_ylim(0, 105) # Set y-axis limit to slightly above 100% for text labels
    ax.legend()
    ax.grid(True, axis='y', linestyle=':', alpha=0.7) # Add a horizontal grid

    plt.xticks(rotation=45, ha="right")
    plt.tight_layout()

    if output_type == 'figure':
        return fig
    elif output_type == 'display':
        plt.show()
        return None
    else:
        print(f"Warning: Unknown output_type '{output_type}'. Displaying plot by default.")
        plt.show()
        return None

## 🤖 Natural Language Processing

In [ ]:
# Helper for weekday calculations
DAYS_OF_WEEK = {
    "monday": 0, "tuesday": 1, "wednesday": 2, "thursday": 3,
    "friday": 4, "saturday": 5, "sunday": 6
}
DAYS_OF_WEEK_LIST = ["monday", "tuesday", "wednesday", "thursday", "friday", "saturday", "sunday"]

def get_days_offset_for_weekday(target_day_name_lower):
    """Calculates the number of days from today until the next occurrence of target_day_name."""
    if target_day_name_lower not in DAYS_OF_WEEK:
        return None

    today_idx = datetime.today().weekday()  # Monday is 0 and Sunday is 6
    target_day_idx = DAYS_OF_WEEK[target_day_name_lower]

    if today_idx == target_day_idx: # Asking for today's day name
        return 0 
    elif target_day_idx > today_idx:
        return target_day_idx - today_idx
    else:  # target_day_idx < today_idx (e.g., today is Wed, target is Mon of next week)
        return 7 - (today_idx - target_day_idx)

# Define parse_weather_question() and generate_weather_response() here
def parse_weather_question(question):
    """
    Parse a natural language weather question.
    """
    original_question = question
    q_lower = question.lower()
    
    # --- Default values ---
    parsed_info = {
        "location": None,
        "time_period_keyword": "today",
        "days_offset": 0,
        "specific_date": datetime.today().strftime("%Y-%m-%d"),
        "attribute": "general",
        "original_question": original_question
    }

    # --- 1. Identify Time Period (same as before) ---
    time_found = False

    # "in X days"
    m_in_days = re.search(r"in\s+(\d+)\s+days?", q_lower)
    if m_in_days:
        num_days = int(m_in_days.group(1))
        if 0 <= num_days <= 5:
            parsed_info["days_offset"] = num_days
            parsed_info["time_period_keyword"] = f"in_{num_days}_days"
            parsed_info["specific_date"] = (datetime.today() + timedelta(days=num_days)).strftime("%Y-%m-%d")
            time_found = True
        else:
            parsed_info["time_period_keyword"] = "future_unsupported"
            parsed_info["days_offset"] = None
            parsed_info["specific_date"] = None
            time_found = True

    # Days of the week
    if not time_found:
        for day_name in DAYS_OF_WEEK_LIST:
            if re.search(rf"\b(on\s+|next\s+)?{day_name}\b", q_lower):
                offset = get_days_offset_for_weekday(day_name)
                if offset is not None and 0 <= offset <= 5:
                    parsed_info["days_offset"] = offset
                    parsed_info["time_period_keyword"] = day_name
                    parsed_info["specific_date"] = (datetime.today() + timedelta(days=offset)).strftime("%Y-%m-%d")
                    time_found = True
                    break
                elif offset is not None and offset > 5:
                    parsed_info["time_period_keyword"] = "future_unsupported"
                    parsed_info["days_offset"] = None
                    parsed_info["specific_date"] = None
                    time_found = True
                    break

    # "Tomorrow"
    if not time_found and "tomorrow" in q_lower:
        parsed_info["days_offset"] = 1
        parsed_info["time_period_keyword"] = "tomorrow"
        parsed_info["specific_date"] = (datetime.today() + timedelta(days=1)).strftime("%Y-%m-%d")
        time_found = True

    # "Today", "now", "currently", "current"
    if not time_found and any(kw in q_lower for kw in ["today", "now", "currently", "current"]):
        parsed_info["days_offset"] = 0
        parsed_info["time_period_keyword"] = "today"
        parsed_info["specific_date"] = datetime.today().strftime("%Y-%m-%d")
        time_found = True

    # --- 2. Identify Weather Attribute ---
    attribute_keywords_map = {
        "temperature": [r"temperature", r"temp\b", r"hot", r"cold", r"warm", r"degrees"],
        "rain": [r"rain", r"raining", r"precipitation", r"showers", r"drizzle", r"wet"],
        "wind": [r"wind", r"windy", r"breeze", r"gale"],
        "humidity": [r"humidity", r"humid", r"damp"],
        "description": [r"sun", r"sunny", r"clouds?", r"cloudy", r"clear", r"overcast", r"conditions", r"outlook", r"like\b"],
    }

    attribute_found = False
    for attr, patterns in attribute_keywords_map.items():
        for pattern in patterns:
            if re.search(rf"\b{pattern}\b", q_lower):
                parsed_info["attribute"] = attr
                attribute_found = True
                break
        if attribute_found:
            break
    
    if not attribute_found and "weather" in q_lower and parsed_info["attribute"] == "general":
        parsed_info["attribute"] = "description"

    # --- 3. IMPROVED Location Extraction ---
    # Use a more targeted approach to extract location
    location_patterns = [
        # "in [location]" pattern
        r"in\s+([A-Za-z\s]+?)(?:\s+(?:today|tomorrow|yesterday|on\s+\w+|in\s+\d+\s+days?)|\?|$)",
        # "for [location]" pattern  
        r"for\s+([A-Za-z\s]+?)(?:\s+(?:today|tomorrow|yesterday|on\s+\w+|in\s+\d+\s+days?)|\?|$)",
        # "[location] weather/temperature" pattern
        r"([A-Za-z\s]+?)\s+(?:weather|temperature|temp|forecast)",
        # "weather in [location]" pattern
        r"weather\s+in\s+([A-Za-z\s]+?)(?:\s+(?:today|tomorrow|yesterday|on\s+\w+|in\s+\d+\s+days?)|\?|$)",
    ]
    
    extracted_location = None
    for pattern in location_patterns:
        match = re.search(pattern, question, re.IGNORECASE)
        if match:
            potential_location = match.group(1).strip()
            # Clean up the location
            potential_location = re.sub(r'\b(the|weather|temperature|temp|will|be|is|what|how)\b', '', potential_location, flags=re.IGNORECASE)
            potential_location = ' '.join(potential_location.split())  # normalize spaces
            
            if potential_location and potential_location.lower() not in ["here", "my location", "current location", "me"]:
                extracted_location = potential_location.title()  # Capitalize properly
                break
    
    # Fallback: try to extract any capitalized words that might be places
    if not extracted_location:
        # Look for sequences of capitalized words (likely place names)
        cap_words = re.findall(r'\b[A-Z][a-z]+(?:\s+[A-Z][a-z]+)*\b', question)
        for word_seq in cap_words:
            if word_seq.lower() not in ['what', 'will', 'today', 'tomorrow', 'weather', 'temperature']:
                extracted_location = word_seq
                break
    
    parsed_info["location"] = extracted_location
    
    return parsed_info

## 🧭 User Interface

In [ ]:
# Define menu functions using pyinputplus or ipywidgets here

def display_current_weather(current_data):
    """Neatly displays current weather conditions."""
    if not current_data:
        print("No current weather data to display.")
        return
    print("\n--- Current Weather ---")
    print(f"  Temperature: {current_data.get('temperature_c', 'N/A')}°C")
    print(f"  Feels Like:  {current_data.get('feels_like_c', 'N/A')}°C")
    print(f"  Condition:   {current_data.get('description', 'N/A')}")
    print(f"  Humidity:    {current_data.get('humidity_percent', 'N/A')}%")
    print(f"  Wind:        {current_data.get('wind_kmph', 'N/A')} km/h")
    print(f"  Precipitation: {current_data.get('precipitation_mm', 'N/A')} mm")




def display_forecast(forecast_list):
    """Neatly displays weather forecast."""
    if not forecast_list:
        print("No forecast data to display.")
        return
    print("\n--- Forecast ---")
    for i, day_fc in enumerate(forecast_list):
        print(f"\n  Day {i+1} ({day_fc.get('date', 'N/A')}):")
        print(f"    Max Temp:    {day_fc.get('max_temp_c', 'N/A')}°C")
        print(f"    Min Temp:    {day_fc.get('min_temp_c', 'N/A')}°C")
        print(f"    Avg Temp:    {day_fc.get('avg_temp_c', 'N/A')}°C")
        print(f"    Condition:   {day_fc.get('description', 'N/A')}")
        print(f"    Rain Chance: {day_fc.get('chance_of_rain_percent', 'N/A')}%")
        print(f"    Precipitation: {day_fc.get('total_precip_mm', 'N/A')} mm")



def handle_visualisations(weather_data_for_viz, location_name):
    """Asks user if they want to see visualisations and displays them."""
    if not weather_data_for_viz or weather_data_for_viz.get("error") or not weather_data_for_viz.get("forecast"):
        print("Not enough data for visualisations.")
        return

    # Initial prompt to see any visualisations
    show_viz = input(f"\nWould you like to see visualisations for {location_name}? (yes/no): ").strip().lower()
    if show_viz in ['no', 'n']:
        return # User doesn't want any visualisations

    while True: # This loop allows asking for different types of visualisations
        try:
            print("\nWhich visualisation would you like to see?")
            print("1. Temperature Trends")
            print("2. Precipitation Forecast")
            print("3. Both")
            print("4. Done with visualisations")
            
            choice = input("Enter your choice (1-4): ").strip()

            if choice == "1":
                print(f"\nGenerating temperature visualisation for {location_name}...")
                try:
                    create_temperature_visualisation(weather_data_for_viz, output_type='display')
                    print("Temperature visualization completed.")
                    
                    # Force output flush and small delay after matplotlib
                    import sys
                    import time
                    sys.stdout.flush()
                    sys.stderr.flush()
                    time.sleep(0.5)  # Short delay to let matplotlib finish
                    
                except Exception as e:
                    print(f"Error creating temperature visualization: {e}")
                    import traceback
                    traceback.print_exc()
                    
            elif choice == "2":
                print(f"\nGenerating precipitation visualisation for {location_name}...")
                try:
                    create_precipitation_visualisation(weather_data_for_viz, output_type='display')
                    print("Precipitation visualization completed.")
                    
                    # Force output flush and small delay after matplotlib
                    import sys
                    import time
                    sys.stdout.flush()
                    sys.stderr.flush()
                    time.sleep(0.5)  # Short delay to let matplotlib finish
                    
                except Exception as e:
                    print(f"Error creating precipitation visualization: {e}")
                    import traceback
                    traceback.print_exc()
                    
            elif choice == "3":
                print(f"\nGenerating temperature visualisation for {location_name}...")
                try:
                    create_temperature_visualisation(weather_data_for_viz, output_type='display')
                    print("Temperature visualization completed.")
                    
                    # Force output flush after first chart
                    import sys
                    import time
                    sys.stdout.flush()
                    sys.stderr.flush()
                    time.sleep(0.5)
                    
                except Exception as e:
                    print(f"Error creating temperature visualization: {e}")
                    
                print(f"\nGenerating precipitation visualisation for {location_name}...")
                try:
                    create_precipitation_visualisation(weather_data_for_viz, output_type='display')
                    print("Precipitation visualization completed.")
                    
                    # Force output flush after second chart
                    sys.stdout.flush()
                    sys.stderr.flush()
                    time.sleep(0.5)
                    
                except Exception as e:
                    print(f"Error creating precipitation visualization: {e}")
                    
            elif choice == "4":
                print("Okay, no more visualisations for this location.")
                break # Exit the while loop for visualisations
            else:
                print("Invalid choice. Please enter 1, 2, 3, or 4.")
                continue
            
            # After showing a visualization, ask if they want to see another
            print("\n" + "="*60)
            print("🔄 VISUALIZATION MENU OPTIONS")
            print("="*60)
            
            # Force everything to flush before asking for input
            import sys
            sys.stdout.flush()
            sys.stderr.flush()
            
            # Try multiple input methods for better compatibility
            try:
                print("See another type of visualisation for this location?")
                print("   Type 'yes' or 'no' and press Enter:")
                sys.stdout.flush()  # Force output to display
                another_viz = input().strip().lower()
            except:
                # Fallback method
                print("See another type of visualisation for this location? (yes/no)")
                another_viz = input().strip().lower()
            
            if another_viz in ['no', 'n', '']:
                print("Exiting visualisation menu...")
                break # Exit the while loop for visualisations
            elif another_viz in ['yes', 'y']:
                print("Showing visualisation menu again...")
                continue  # Continue the loop to show menu again
            else:
                print("Invalid response, assuming 'no'...")
                break
                
        except (EOFError, KeyboardInterrupt):
            print("\nInput interrupted, returning to main menu...")
            break
        except Exception as e:
            print(f"Unexpected error in visualization menu: {e}")
            import traceback
            traceback.print_exc()
            break

    print(f"\nFinished with visualisations for {location_name}.")

## 🧩 Main Application Logic

In [ ]:
# ===================================================================
# MAIN APPLICATION LOGIC
# ===================================================================
# Core functionality that ties everything together

def generate_weather_response(parsed_question, weather_data):
    """
    Generate a natural language response using AI for weather questions.
    
    Args:
        parsed_question (dict): Parsed question data
        weather_data (dict): Weather data
        
    Returns:
        str: Natural language response
    """
    if weather_data.get("error"):
        return f"Sorry, I couldn't get weather data. Error: {weather_data['error']}"

    # Prepare context for the AI
    location = parsed_question.get("location", "the requested location")
    time_period = parsed_question.get("time_period_keyword", "today")
    attribute = parsed_question.get("attribute", "general")
    original_question = parsed_question.get("original_question", "")
    days_offset = parsed_question.get("days_offset", 0)
    
    # Build context string for AI
    context = f"Original question: {original_question}\n\n"
    
    # Add current weather if available
    if weather_data.get("current"):
        current = weather_data["current"]
        context += f"Current weather in {location}:\n"
        context += f"- Temperature: {current['temperature_c']}°C\n"
        context += f"- Feels like: {current['feels_like_c']}°C\n"
        context += f"- Conditions: {current['description']}\n"
        context += f"- Humidity: {current['humidity_percent']}%\n"
        context += f"- Wind: {current['wind_kmph']} km/h\n"
        context += f"- Precipitation: {current['precipitation_mm']} mm\n\n"
    
    # Add forecast data if available
    if weather_data.get("forecast"):
        context += f"Weather forecast for {location}:\n"
        for i, day in enumerate(weather_data["forecast"]):
            # Determine what day this represents
            if i == 0:
                day_label = "Today"
            elif i == 1:
                day_label = "Tomorrow"
            else:
                day_label = f"Day {i+1}"
            
            # Mark the specifically requested day
            if days_offset == i:
                day_label += " ⭐ (REQUESTED DAY)"
            
            context += f"{day_label} ({day['date']}):\n"
            context += f"  - Temperature: High {day['max_temp_c']}°C, Low {day['min_temp_c']}°C, Average {day['avg_temp_c']}°C\n"
            context += f"  - Weather: {day['description']}\n"
            context += f"  - Chance of rain: {day['chance_of_rain_percent']}%\n"
            context += f"  - Expected precipitation: {day['total_precip_mm']} mm\n\n"
    
    # Create AI prompt based on what they're asking for
    if attribute == "temperature":
        focus_instruction = f"Focus specifically on temperature information for {time_period}."
    elif attribute == "rain":
        focus_instruction = f"Focus specifically on precipitation/rain information for {time_period}."
    elif attribute == "wind":
        focus_instruction = f"Focus specifically on wind conditions for {time_period}."
    elif attribute == "humidity":
        focus_instruction = f"Focus specifically on humidity levels for {time_period}."
    elif attribute == "description":
        focus_instruction = f"Focus on general weather conditions and outlook for {time_period}."
    else:
        focus_instruction = f"Provide a helpful general weather summary for {time_period}."

    prompt = f"""You are a friendly and knowledgeable weather assistant. Answer the user's weather question in a natural, conversational way.

{context}

Instructions:
- Answer the original question directly and specifically
- {focus_instruction}
- Be conversational and helpful, not robotic
- If they asked about a specific day, make sure to highlight that day's information
- Include relevant context but don't overwhelm with unnecessary details
- Use natural language, as if you're talking to a friend

Please provide a natural response to their question:"""

    try:
        # Use the AI model to generate natural response
        ai_response = get_response(prompt)
        return ai_response.strip()
    except Exception as e:
        # Fallback to enhanced template response if AI fails
        print(f"Note: AI response unavailable ({e}), using fallback response.")
        return generate_enhanced_fallback_response(parsed_question, weather_data)

def generate_enhanced_fallback_response(parsed_question, weather_data):
    """
    Enhanced fallback template-based response if AI fails.
    """
    location = parsed_question.get("location", "the location")
    time_period = parsed_question.get("time_period_keyword", "today")
    attribute = parsed_question.get("attribute", "general")
    days_offset = parsed_question.get("days_offset", 0)
    
    # Handle current weather (today)
    if weather_data.get("current") and days_offset == 0:
        current = weather_data["current"]
        if attribute == "temperature":
            return f"The current temperature in {location} is {current['temperature_c']}°C, and it feels like {current['feels_like_c']}°C. {current['description']} conditions right now."
        elif attribute == "rain":
            if current['precipitation_mm'] > 0:
                return f"It's currently {current['description'].lower()} in {location} with {current['precipitation_mm']}mm of precipitation."
            else:
                return f"No rain currently in {location}. Conditions are {current['description'].lower()}."
        elif attribute == "wind":
            return f"Current wind conditions in {location}: {current['wind_kmph']} km/h. Temperature is {current['temperature_c']}°C with {current['description'].lower()} skies."
        elif attribute == "humidity":
            return f"Current humidity in {location} is {current['humidity_percent']}%. Temperature: {current['temperature_c']}°C, conditions: {current['description'].lower()}."
        else:
            return f"Current weather in {location}: {current['temperature_c']}°C (feels like {current['feels_like_c']}°C), {current['description'].lower()}, humidity {current['humidity_percent']}%, wind {current['wind_kmph']} km/h."
    
    # Handle forecast data (tomorrow and beyond)
    elif weather_data.get("forecast") and len(weather_data["forecast"]) > days_offset:
        forecast_day = weather_data["forecast"][days_offset]
        day_name = "tomorrow" if days_offset == 1 else time_period
        
        if attribute == "temperature":
            return f"For {day_name} in {location}, expect a high of {forecast_day['max_temp_c']}°C and a low of {forecast_day['min_temp_c']}°C. Conditions should be {forecast_day['description'].lower()}."
        elif attribute == "rain":
            if forecast_day['chance_of_rain_percent'] > 50:
                return f"There's a {forecast_day['chance_of_rain_percent']}% chance of rain {day_name} in {location}. Expect {forecast_day['description'].lower()} conditions with possible precipitation of {forecast_day['total_precip_mm']}mm."
            else:
                return f"Low chance of rain {day_name} in {location} ({forecast_day['chance_of_rain_percent']}%). Should be {forecast_day['description'].lower()} with temperatures between {forecast_day['min_temp_c']}°C and {forecast_day['max_temp_c']}°C."
        else:
            return f"Weather {day_name} in {location}: {forecast_day['max_temp_c']}°C/{forecast_day['min_temp_c']}°C, {forecast_day['description'].lower()}, {forecast_day['chance_of_rain_percent']}% chance of rain."
    
    return f"I have weather data for {location}, but couldn't find specific information for {time_period} regarding {attribute}."

def handle_get_weather_direct():
    """Handles the 'Get weather by location' menu option."""
    print("\n--- Get Weather by Location ---")
    location = pyip.inputStr("Enter the city or location name: ")
    if not location:
        print("Location cannot be empty.")
        return

    forecast_days = pyip.inputInt("Enter number of forecast days (1-5, default is 3): ", min=1, max=5, default=3, blank=True)
    if forecast_days is None: # If user just presses Enter for blank=True
        forecast_days = 3

    print(f"\nFetching weather data for {location} for {forecast_days} day(s)...")
    weather_data = get_weather_data(location, forecast_days=forecast_days)

    if weather_data.get("error"):
        print(f"\nError: {weather_data['error']}")
        return

    display_current_weather(weather_data.get("current"))
    display_forecast(weather_data.get("forecast"))
    
    # Offer visualisations
    handle_visualisations(weather_data, location)

def handle_ask_weather_question():
    """Enhanced weather question handler with better messaging and AI integration."""
    print("\n--- Ask a Weather Question ---")
    question = pyip.inputStr("Ask your weather question (e.g., 'What's the temperature in London tomorrow?'):\n")
    if not question:
        print("Question cannot be empty.")
        return

    parsed_q = parse_weather_question(question)
    print(f"\nParsed Question: {parsed_q}")  # For debugging - you can remove this later

    location = parsed_q.get("location")
    if not location:
        location = pyip.inputStr("I couldn't determine a location from your question. Please enter the city/location: ")
        if not location:
            print("Location is required to get weather.")
            return
        parsed_q["location"] = location

    # Handle unsupported future dates
    if parsed_q.get("days_offset") is None and parsed_q.get("time_period_keyword") == "future_unsupported":
        print(f"Sorry, I can't provide a forecast for dates that far out for {location}.")
        return
        
    # Calculate forecast days needed
    forecast_days_needed = max(1, (parsed_q.get("days_offset", 0) + 1))
    api_forecast_days = min(forecast_days_needed, 5)

    # Better messaging about what we're fetching
    time_keyword = parsed_q.get("time_period_keyword", "today")
    if time_keyword == "today":
        print(f"\nFetching current weather for {location}...")
    elif time_keyword == "tomorrow":
        print(f"\nFetching weather forecast for {location} (including tomorrow's forecast)...")
    elif "day" in time_keyword.lower():
        print(f"\nFetching weather forecast for {location} ({time_keyword})...")
    else:
        print(f"\nFetching weather forecast for {location} (up to {api_forecast_days} days)...")

    # Get weather data
    weather_data = get_weather_data(location, forecast_days=api_forecast_days)

    if weather_data.get("error"):
        print(f"\nError: {weather_data['error']}")
        return

    # Generate AI-powered natural language response
    print("\n--- Weather Advisor Says ---")
    response = generate_weather_response(parsed_q, weather_data)
    print(response)
    
    # Offer visualizations if we have forecast data
    if weather_data.get("forecast") and len(weather_data["forecast"]) > 0:
        handle_visualisations(weather_data, location)

def main_menu():
    """Displays the main menu and handles user choices."""
    print("Welcome to the Weather Advisor!")
    while True:
        print("\n--- Main Menu ---")
        choice = pyip.inputMenu(
            ["Get weather by location (Direct)",
             "Ask a weather question (Natural Language)",
             "Exit"],
            prompt="What would you like to do?\n",
            numbered=True
        )

        if choice == "Get weather by location (Direct)":
            handle_get_weather_direct()
        elif choice == "Ask a weather question (Natural Language)":
            handle_ask_weather_question()
        elif choice == "Exit":
            print("Thank you for using Weather Advisor. Goodbye!")
            break
        
        input("\nPress Enter to return to the main menu...") # Pause before re-displaying menu



In [ ]:
# ===================================================================
#  APPLICATION STARTUP
# ===================================================================

main_menu()

## 🧪 Testing and Examples

In [ ]:
# Include sample input/output for each function

## 🗂️ AI Prompting Log (Optional)
Add markdown cells here summarising prompts used or link to AI conversations in the `ai-conversations/` folder.